In [1]:
import time
import re
import sys
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [127]:
def scraper(asin):
    
    df = pd.DataFrame(columns=['rating', 'content'])
    pages = range(1,3)
    asin=asin
    
    for page in pages:
        
        # iterable url
        url = 'https://www.amazon.com/product-reviews/{}/ref=cm_cr_getr_d_paging_btm_prev_1?pageNumber={}'.format(asin, page)
        
        # getting soup
        driver = webdriver.Chrome()
        time.sleep(1)
        driver.get(url)
        page_source = driver.page_source
        driver.quit()
        soup = BeautifulSoup(page_source,'lxml')
        
        # parsing soup
        reviews = soup.findAll("div", {"class":"a-section review aok-relative"})
        ## parsing reviews section
        reviews = BeautifulSoup('<br/>'.join([str(tag) for tag in reviews]), 'html.parser')
        
        #getting content
        contents = reviews.find_all("span", {"data-hook":"review-body"})
        content_lst = []
        for content in contents:
            text_ = content.find_all("span")[0].get_text("\n").strip()
            text_ = ". ".join(text_.splitlines())
            text_ = re.sub(' +', ' ', text_)
            content_lst.append(text_)
        print(len(content_lst))
            
        #getting rating
        ratings = reviews.find_all("i", {"data-hook":"review-star-rating"})
        full_rating_lst = []
        for rating in ratings:
            full_rating_lst.append(rating.find_all("span")[0].contents[0])
            
        
        rating_lst = []
        for rating in full_rating_lst:
            rating_lst.append(re.findall("\d+\.\d+", rating)[0])
        print(len(rating_lst))
            
            
        # concatenating to main data frame
        
        temp_df = pd.DataFrame({'rating':rating_lst, 'content':content_lst})
        df = df.append(temp_df)
        
    return df


In [128]:
scraper('B08WWFWRY6')

10
10
10
10


,rating,content
0,3.0,Almost a decade old game (as of this review) n...
1,1.0,"The controls for this game are so bad, it make..."
2,1.0,UPDATE : a few days later..... I'm going to fl...
3,5.0,If you really liked Breath of the Wild this on...
4,2.0,The controls were aweful on the pro controller...
5,5.0,The game was 4 stars on the Wii because of of ...
6,4.0,"As a long-time Zelda fan, I've been itching fo..."
7,3.0,Played it for a few hours yesterday and compar...
8,5.0,This game is pure adventure. It plays like the...
9,4.0,"Okay, so, caveats first:. I've played through ..."
